In [19]:
import glob
import pandas
import pandas as pd
import numpy as np
import os
import time
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
from nltk.stem.porter import *
# from sklearn.neighbors import KNeighborsClassifier

stemmer = PorterStemmer()



os.chdir('/Users/krothaps/Desktop/hackathon/dataset/tim_first_level_final/')

In [2]:
def inlcude_only_nouns(lines):
    # function to test if something is a noun
    is_noun = lambda pos: pos[:2] == 'NN'

    tokenized = nltk.word_tokenize(lines)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 

    return ' '.join(nouns)

In [3]:
def parse_file_data(file_name):
    with open(filename) as f:
        content = f.readlines()
        
    shortword = re.compile(r'\W*\b\w{1,3}\b')
    
    file_text_data = ''
    for line in content:
        line = line.decode('utf-8')
        text_data = "\t".join(line.split("\t", 2)[2:])
        # include only nouns
        text_data = inlcude_only_nouns(text_data)
        # remove words which are 3 characters or less
        text_data = shortword.sub('', text_data)
        # remove urls
        text_data = re.sub(r'http\S+', '', text_data)
        text_data = re.sub(r'https\S+', '', text_data)
#         text_data = ''.join([i for i in text_data if not i.isdigit()])
        # lemmatization
#         text_data = ' '.join([lmtzr.lemmatize(word) for word in text_data.split()])
        # stemming        
#         text_data = ' '.join([stemmer.stem(word) for word in text_data.split()])
        # spell correction
#         text_data = text_data.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('  ', ' ')
        text_data = text_data.lower()
        #include only numerical data
        text_data = re.sub(r"[^a-z]+", ' ', text_data)
        
        file_text_data += text_data
        
    return filename+'\t'+file_text_data

In [4]:
import time

start_time = time.time()

output = open('/Users/krothaps/Desktop/hackathon/dataset/test_final.txt', 'w')

count = 0

for filename in glob.glob(os.path.join('*.txt')):
    count += 1
    start1_time = time.time()
    file_data = parse_file_data(filename)
    output.write(file_data+'\n')
    if count % 10 == 0 :
        print str(count)+'\t'+str(int(time.time() - start1_time))
    
output.close()

print "--- Finished parsing text in " + str(time.time() - start_time)+' seconds---'


10	2
20	14
30	10
40	7
50	7
60	5
70	7
80	0
90	4
100	1
110	7
120	0
130	8
140	0
150	6
160	14
170	10
180	0
190	0
200	2
210	1
220	1
230	8
240	0
250	12
260	6
270	11
280	1
290	0
300	0
310	3
320	4
330	0
340	27
350	0
360	0
370	0
380	0
390	0
400	0
410	1
420	3
430	8
440	0
450	1
460	6
470	0
480	5
490	3
500	4
510	0
520	8
530	1
540	1
550	2
560	10
570	2
580	0
590	1
600	0
610	0
620	1
630	5
640	3
650	9
660	0
670	0
680	0
690	0
700	0
710	0
720	0
730	3
740	3
750	3
760	0
770	0
780	0
790	1
800	0
810	5
820	3
830	0
840	0
850	7
860	2
870	2
880	5
890	10
900	6
910	0
920	0
930	0
940	22
950	1
960	5
970	3
980	253
--- Finished parsing text in 6025.00171018 seconds---


In [5]:
# Convert the text data to TF-IDF

In [6]:
print "--- Finished parsing text in " + str(time.time() - start_time)+' seconds---'

--- Finished parsing text in 6025.03002501 seconds---


In [9]:
data = pd.read_csv('/Users/krothaps/Desktop/hackathon/dataset/test_final.txt', delimiter='\t', names=['id','text'])

print data.shape

# tfv = TfidfVectorizer(min_df=250, 
#             strip_accents='unicode', analyzer='word',
#             stop_words = 'english')

tfv = CountVectorizer(max_df=0.95, min_df=5,
                                stop_words='english')

#   ngram_range=(1, 3), token_pattern=r'\w{1,}', use_idf=1,smooth_idf=1,sublinear_tf=1,

df = data['text'].fillna('')

tfv.fit(df)
data_tfv =  tfv.transform(df) 

(978, 2)


(978, 2)


In [10]:
result =  pd.DataFrame(data_tfv.toarray())

# result = result.drop(['id'], axis=1)

result.columns = tfv.get_feature_names()
print len(result.columns)


28835


In [61]:
print result.shape

result_filtered = result
sum_colums = []

for column in result:
    if sum(result[column]) < 500:
        sum_colums.append(column)
        
print 'done'        
result_filtered = result_filtered.drop(sum_colums, axis=1)

print result_filtered.shape




result_filtered.to_csv('count_vectorizer_min_500.txt', sep='\t')

(978, 28835)
done
(978, 3695)


In [69]:
print data.columns

result_filtered_final = pd.concat([pd.DataFrame(data['id']), result_filtered], axis=1)

print result_filtered_final.shape
result_filtered_final.to_csv('count_vectorizer_min_500_v1.txt', sep='\t', index=False)

Index([u'id', u'text'], dtype='object')
(978, 3696)


In [94]:
# df1 = final[['id','computing']]

# print df1.shape

# df1.to_csv('two_column_output.txt', sep='\t')



In [83]:
# Distance Matrix 

import pandas as pd
from scipy.spatial import distance_matrix


print type(data)

tfidf_scores = final.drop(['id'], axis=1)

tfidf_scores_list = tfidf_scores.values.tolist()

id_list = final.ix[:,0].tolist()


<class 'pandas.core.frame.DataFrame'>


/Users/krothaps/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


In [95]:
print result.shape
print data.shape


final = pd.concat([data, result], axis=1)

print final.shape

final = final.drop(['text'], axis=1)

print final.shape

sl = pd.Series(school_list)

final['school_name'] = sl.values

final.to_csv('sample_final_count_vectorizer.txt', sep='\t')


(979, 28837)
(979, 2)
(979, 28839)
(979, 28837)


In [124]:
feature = 'analy'

for x in result.columns:
    if feature in x:
        print x

analys
analyses
analysis
analyst
analysts
analytic
analytical
analytics
analyze
analyzer
analyzes
analyzing
psychoanalysis


In [125]:
# count vectorizer data

feature = 'applied'

# count_vectorizer_final = final

eda = count_vectorizer_final[['school_name', feature]]

print eda.sort_values(by=feature, ascending=False).head(30)


                                       school_name  applied
491                     kean college of new jersey     4183
537                               le moyne college     1224
512                  western new mexico university      629
221                      lewis-clark state college      452
237               illinois institute of technology      381
174                south florida community college      324
762                        dakota state university      321
798           texas a & m international university      309
450                       missouri baptist college      243
666             oklahoma state university-okmulgee      242
790                texas a & m university-commerce      241
858  virginia polytechnic institute and state univ      220
464                          washington university      197
217              university of hawaii at west oahu      162
391                                   hope college      156
874                     highline communi

In [84]:
df_uid_school_name = pd.read_csv('/Users/krothaps/Desktop/hackathon/presentation/uid_school_name.txt',  names = ['uid', 'school_name'])

# convert to lower case 
df_uid_school_name['school_name']  =  df_uid_school_name['school_name'].str.lower()
# remove duplicates
df_uid_school_name = df_uid_school_name.drop_duplicates()


print df_uid_school_name.shape

# print df_uid_school_name.dtypes

df_uid_school_name['uid'] = df_uid_school_name['uid'].astype(str)

school_list = []


for x in id_list:
    s = x.replace('.txt', '')
    try:
        matching_row = df_uid_school_name.loc[df_uid_school_name['uid'] == s].iloc[0]
        school_list.append(matching_row['school_name'])
    except:
        school_list.append('missing')

df = pd.DataFrame(tfidf_scores_list, columns=tfidf_scores.columns.tolist(), index=school_list)        
        
distance_matrix = pd.DataFrame(distance_matrix(df.values, df.values), index=df.index, columns=df.index)

print distance_matrix.shape

(17287, 2)
(979, 979)


In [85]:
distance_matrix.to_csv('/Users/krothaps/Desktop/hackathon/presentation/distance_matrix2.txt', sep='\t')
